In [2]:
import geopandas as gpd
import pandas as pd
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from config import settings

In [2]:
#BUCKET_NAME = 'ds-interview-sandbox'
#client = boto3.client('s3')
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(BUCKET_NAME)

In [3]:
gdf = gpd.read_file('./uga_regions.geojson')
gdf.columns = [ x.lower() for x in gdf.columns]

In [6]:
#s3 = boto3.resource('s3')
#bucket = s3.Bucket('ds-interview-sandbox')
# Iterates through all the objects, doing the pagination for you. Each obj
# is an ObjectSummary, so it doesn't contain the body. You'll need to call
# get to get the whole body.
#for obj in bucket.objects.filter(Prefix='shared/ug'):
    #key = obj.key
    #print(key)

In [7]:
#for obj in bucket.objects.filter(Prefix='shared/ug'):
    #key = obj.key
    #s3.Bucket(BUCKET_NAME).download_file(key, f"./{key.split('/')[-1]}")
    #gdf = gpd.read_file(f"./{key.split('/')[-1]}")
    #gdf.to_postgis()
    #print(gdf.shape)

In [7]:
DATABASE_URL = f'postgresql+psycopg2://{settings.database_username}:{settings.database_password}@{settings.database_host}:{settings.database_port}/{settings.database_name}'
engine = create_engine(DATABASE_URL)

In [21]:
df = pd.read_csv("./ug_clusters.csv")
ug_clusters = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGNUM, df.LATNUM))
ug_clusters.columns = [x.lower() for x in ug_clusters.columns]
ug_clusters = ug_clusters.set_crs(4326, allow_override=True)
ug_clusters.to_postgis("ug_clusters", engine)

In [31]:
df = pd.read_csv("./uga_dhs_2016.csv")
df.columns = [x.lower() for x in df.columns]
df.to_sql("ug_dhs", engine)

842

In [29]:
gdf = gpd.read_file("./uga_regions.geojson")
gdf.columns = [x.lower() for x in gdf.columns]
gdf.to_postgis("ug_regions", engine)

In [12]:
query = text("""SELECT ug_dhs.v751, ug_dhs.v001, ug_clusters.latnum, ug_clusters.longnum, ug_dhs.sex, ug_regions.geometry  
FROM ug_dhs 
JOIN ug_clusters ON ug_dhs.v001 = ug_clusters.v001
JOIN ug_regions ON ug_regions.name = 'Northern'
WHERE ug_dhs.sex = 'male'""")

In [4]:
query = text("select count(dhs.sex) as count from ug_dhs dhs join (select a.v001, (case when ST_contains(b.geometry, a.geometry) then b.name else 'n/a' end) as region FROM public.ug_clusters a, public.ug_regions b) regions on dhs.v001=regions.v001 where regions.region=:region and dhs.sex=:sex and dhs.v751=:aids").bindparams(region='Northern', sex='male', aids=1)


In [88]:
region = "'Northern'"
sex = "'Male'"
aids = 1

In [89]:
#query = "select count(dhs.sex) as count from ug_dhs dhs join (select a.v001, (case when ST_contains(b.geometry, a.geometry) then b.name else 'n/a' end) as region FROM public.ug_clusters a, public.ug_regions b) regions on dhs.v001=regions.v001 where regions.region={region} and dhs.sex={sex} and dhs.v751={aids}"


In [8]:
engine.execute(query).fetchone()[0]


1232